# Fórmula de Black-Scholes para opciones Europeas.

## Fórmula de Black-Scholes para opciones de compra Europeas.

Supongamos que el valor de un activo subyacente en el instante $t$, $S_t$, viene descrito mediante el movimiento Browniano geométrico, es decir:
$$ dS(t) = \mu S(t)dt+\sigma S(t) dW(t), $$
cuya solución podemos calcular aplicando el Lema de Itô. Supongamos que tiene un valor en $t=0$ de $s_0$, esto es $S(0) = s_0$. Ahora podemos tomar $A^{(1)}(t,S(t))= \mu S(t), \ A^{(2)}(t,S(t))= \sigma S(t).$ Si tomamos $s=0,$ y $f(t,s)=log(s)$ podemos calcular las derivadas parciales en $t$ y $s$, tenemos:
$$ f_t(t,s) = 0, \ \ \ f_s(t,s) = \frac{1}{s}, \ \ \ f_{s^2}(t,s) = \frac{-1}{s^2}.$$

Si aplicamos el Lema de Itô, obtenemos:

$$ log(S(t)) - log(S(0)) = \int_{0}^{t} \mu S(y) \frac{1}{S(y)} dy + \frac{1}{2} \int_{0}{t} \sigma ^2 (S(y))^2 \left ( -\frac{1}{S(y)^2} \right) dy + \int_{0}^{t} \sigma ^2 (S(y))^2 \left ( -\frac{1}{S(y)^2} \right) dW(y) = \mu t - \frac{1}{2} \sigma ^2 t + \sigma ^2 (W(t) - W(0)).$$

Por tanto:

$$ log \left ( \frac{S(t)}{s_0} \right ) = \left ( \mu - \frac{1}{2} \sigma ^2 \right) t + \sigma W(t). $$

Si depejamos $S(t)$ en la ecuacion anterior, se tiene:

$$  S(t) = s_0 e^{ (\mu -\frac{1}{2} \sigma ^2)t + \sigma W(t)} .$$

Ahora suponemos que tenemos un activo subyacente cuyo valor se modeliza mediante un movimiento Browniano geométrico como el expuesto anteriormente. Si se emite una opción de compra Europea sobre este activo con fecha de vencimiento $T$, el precio del ejercicio $K$. Denotamos por $C$ el precio de la opción $call$ (prima). Según la fórmula de Black-Scholes, este precio viene dado por:

$$ C = s_0 N(d_1) - K e^{-r (T - T_0)} N(d_2).$$

Donde $r$ es el tipo de interés libre de riesgo, $N(x)$ es la función de distribución de una gaussiana tipificada $N(0,1)$, es decir,
$$ N(x) = \mathbb{P}[Z \leq x] = \frac{1}{2 \pi } \int_{- \infty }^{x} e^{- \frac{1}{2} s^2 } ds, $$
y
$$ d_1 = \frac{log \left( \frac{s_0}{K} \right) + \left( r + \frac{ \sigma ^2 }{2} \right) (T - T_0)}{ \sigma \sqrt{T-T_0} } ,$$
$$ d_2 = d_1 - \sigma \sqrt{T - T_0}.$$

Podemos observar que el valor de la prima solo depende del parámetro $\sigma $ del modelo log-normal.

In [5]:
def valorCall(s0,K,r,sigma,T,T0 = 0):
    """
    La siguiente función nos devuelve el valor de una Call Europea para fecha de vencimiento T:
    Input:
        - s0(Number)            : Valor inicial de la Call.
        - K(Number)             : Precio del ejercicio.
        - r(Number)             : Tipo de interés libre de riesgo.
        - sigma (Number)        : riesgo o volatilidad del activo.
        - T (Number)            : Fecha de vencimiento.
        - T0 = 0 (Number)       : Fecha inicial para el velor S0.
    Output:
        - C(Number)             : Valor de la call.
    """
    from math import log,exp,sqrt
    from scipy.stats import norm

    d1 = (log(s0/K) + (r + (sigma**2)/(2)) * (T-T0)) / (sqrt(T-T0)*sigma)
    d2 = d1 - sigma*sqrt(T-T0)

    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)

    C = s0*Nd1 - K*exp(-r*(T-T0))*Nd2
    return C

### Aplicando la fórmula de Black-Scholes para opciones de compra Europeas.

Suponemos que tenemos una opción con precio inicial $s_0 = 74.625$, el precio del ejercicio $K =100$.La fecha de vencimiento es $1.6$ años, el tipo de interés libre de riesgo es $r=0.05$ anual y el riesgo $\sigma = 0.375$ anual.

In [22]:
#Nuestro ejemplo
s0 = 74.625
K = 100
T = 1.6
r = 0.05
sigma = 0.375

C = valorCall(s0,K,r,sigma,T)
C 

8.316364366583233

### Deducción de la fórmula de Black-Scholes de opciones de compra Europeas.

El valor de la prima de una opción de compra europea resulta de actualizar el valor esperado del pay-off de la opción
$$ C = e^{-rT} \cdot \mathbb{E} \left[ \left( S_T - K \right)^+ \right]. $$

Como ya hemos visto el valor de $S_T$ viene dado por el modelo Browniano geométrico:
$$  S_T = s_0 e^{ (\mu -\frac{1}{2} \sigma ^2)t + \sigma W(t)} = s_0 e^{ (\mu -\frac{1}{2} \sigma ^2)t + \sigma \sqrt{T} Z} .$$

Entonces si realizamos $  e^{-rT} \cdot  \left( S_T - K \right)^+ ,$ y aplicamos el operador esperanza, obtenemos:

$$ C = \frac{e^{-rT}}{\sqrt{2 \pi}} \int_{- \infty }^{ \infty }  g(x) \cdot f(x)  dx, $$
donde $g(x)$ es transformación de $f(x)$ función de densidad de la Gaussiana tipificada, es decir, $$ g(x) = max \left ( s_0 e^{ (\mu -\frac{1}{2} \sigma ^2)t + \sigma \sqrt{T} x} - K , 0 \right )$$ y  $$ f(x) = e^{- \frac{x^2}{2}} . $$

Como la función $g(x)$ es $0$ cuando no se ejecuta la opción, entonces deducimos $\hat{x}$ de la siguiente forma:

$$ s_0 e^{ (\mu -\frac{1}{2} \sigma ^2)t + \sigma \sqrt{T} x} - K > 0 \Rightarrow x > \frac{log \left ( \frac{K}{s_0} \right )  - \left ( r - \frac{ \sigma ^2 }{2} \right) T }{ \sigma \sqrt{T}} = \hat{x} .  $$

Entonces es fácil ver que:

$$ C = \frac{e^{-rT}}{\sqrt{2 \pi}} \int_{ \hat{x} }^{ \infty }  \left ( s_0 e^{ (\mu -\frac{1}{2} \sigma ^2)t + \sigma \sqrt{T} x} \right ) \cdot e^{- \frac{x^2}{2}}  dx - \frac{e^{-rT} K}{\sqrt{2 \pi}} \int_{ \hat{x} }^{ \infty }  e^{- \frac{x^2}{2}}  dx = \frac{e^{-rT}}{\sqrt{2 \pi}} \int_{ \hat{x} }^{ \infty }  \left ( s_0 e^{ (\mu -\frac{1}{2} \sigma ^2)t + \sigma \sqrt{T} x} \right ) \cdot e^{- \frac{x^2}{2}}  dx - e^{-rT} K N(- \hat{x}). $$

Operando, se obtiene:

$$ C = \frac{s_0}{\sqrt{2 \pi}} \int_{ \hat{x} }^{ \infty }  e^{- \frac{1}{2} \left (  x - \sigma \sqrt{T} \right )^2  }  dx - e^{-rT} K N(- \hat{x})  = s_0 N(- (\hat{x} - \sigma \sqrt{T}) ) - e^{-rT} K N(- \hat{x}) = s_0 N(\sigma \sqrt{T} - \hat{x}) - e^{-rT} K N(- \hat{x}).$$

De aqui es inmediato ver el valor de $d_1 = \sigma \sqrt{T} - \hat{x} $ y $d_2 = - \hat{x}.$ Si operamos en $d_1$ y $d_2$ obtenemos los valores dados anteriormente.

## Fórmula de pariedad entre opciones de compra y opciones de venta Europeas.
Vamos a presentar la expresión que relaciona las primas de una call y de una put europeas sobre un mismo activo subyacente con misma fecha de vencimiento y mismo precio de ejercicio.

Como la fórmula que vamos a utilizar es para opciones Europeas, tenemos que tener en cuenta que solo se podrá ejecutar a fecha de vencimiento $T$, por lo que debemos distinguir dos casos:

- Si el activo subyacente no paga dividendos durante el periodo de vida de la opción.
- Si el activo subyacento sí que paga dividendos durante el periodo de vida de la opción.


### Subyacente no paga dividendos.
Para deducir el valor, vamos a construir dos carteras A y B. 

**Cartera A:** Supongamos que adquirimos una posición larga en la *Put*, es decir, se paga una prima *P* en el instante $t_0$ y esto le da el derecho a vender el subyacente en el instante de tiempo *T*  al precio de ejercicio *K*. Además, se adquiere una posición corta en la *Call*, es decir, se recibe una prima *C* en el instante $t_0$ a cambio de que quien compra la opción tenga derecho a comprar el subyacente en el intante *T* al precio de ejercicio *K*.

**Cartera B:** Por otro lado, se adquiere una posición corta en la acción sobre la que se emiten opciones con vencimiento *T*, cuyo valor inicial en el instante $t_0$ es $S_0$. Por otro lado, se invierte la cantidad de efectivo $Kt_0$, que si actualizamos al instante $t_0$ y al tipo de interés libre de riesgo $r$ el precio de ejericio de la *Call*.

En el instante $t=t_0$, el valor de ambas carteras será:

**Cartera A**: $P -C$,

**Cartera B**: $Ke^{-r(T-t_0)} - S_0$

En el instante $t=T$, el valor de ambas carteras será:

**Cartera A**: $max(K - S_T,0) - max(S_T-K,0) = K-S_T$,

**Cartera B**: $Ke^{-r(T-t_0)}e^{r(T-t_0)} - S_T = K - S_T$.

Como los valores finales de ambas carteras son iguales, haciendo uso del Principio de Ausencia de Oportunidades de Arbitraje, los valores iniciales deberían de ser el mismo, así se deduce:
$$ P-C = Ke^{-r(T-t_0)} - S_0.$$

### Subyacente paga dividendos.

Si la acción si paga dividendos durante el periodo de vida de la opción, fijándonos en la cartera B, como se ha adquirido una acción prestada (posición corta), cuando se devuelva la acción a su propietario, éste además de recuperar el calor de la acción también querrá el dividiendo que se haya repartido en algún instante de tiempo $t_1 \in [t_0,T].$ Denotamos mediente $d_0$ el valor del dividendo en el instante $t_0$, es decir, el valor actualizado del dividendo que se pasa en $t = t_1$, por lo tanto el valor final de la cartera en $t = T$ será:

**Cartera B:** $K - S_t - d_0e^{-r(t-t_0)}$

Ya no podemos igualarlo, por lo que debemos reconstruir la cartera B, siendo ahora la cantidad de efectivo que se invierte $Ke^{-r(T-t_0)} + d_0$, de esta forma el valor en el instante inicial es:

**Cartera B':** $Ke^{-r(T-t_0)} + d_0 - S_0$

y en el instante $t=T$, será:

**Cartera B':** $(Ke^{-r(T-t_0)+d_0})e^{r(T-t_0)} - (S_T + d_0e^{r(t-t_0)}) = K + d_0e^{r(T-t_0)} - S_T - d_0e^{r(t-t_0)} = K - S_T.$

Entonces haciendo como anteriormente, se deduce:

$$ P-C = Ke^{-r(T-t_0)} + d_0 - S_0.$$

## Fórmula de Black-Scholes para opciones de venta Europeas.

Haciendo uso de la fórmula de pariedad y de la fórmula calculada para opciones de compra, podemos llegar a la fórmula de Black-Scholes opciones de venta Europeas.

Si el activo no paga dividendos y teniendo en cuenta que $N(-x)=1-N(x)$, se tiene:

$$ P = Ke^{-r(T-t_0)} - S_0 + S_0N(d_1) - Ke^{-rT}N(d_2) = Ke^{-r(T-t_0)}(1-N(d_2)) -S_0(1 - N(d_1)) = Ke^{-r(T-t_0)}N(-d_2) - S_0N(-d_1) $$

Si el activo si paga dividendos y teniendo en cuenta que $N(-x)=1-N(x)$, se tiene:

$$ P = Ke^{-r(T-t_0)} + d_0 - S_0 + S_0N(d_1) - Ke^{-rT}N(d_2) = Ke^{-r(T-t_0)}(1-N(d_2)) -S_0(1 - N(d_1)) + d_0 = Ke^{-r(T-t_0)}N(-d_2) - S_0N(-d_1) + d_0 $$

In [3]:
def valorPut(s0,K,r,sigma,T,d0 = 0,T0 = 0):
    """
    La siguiente función nos devuelve el valor de una Put Europea para fecha de vencimiento T:
    Input:
        - s0(Number)            : Valor inicial de la Put.
        - K(Number)             : Precio del ejercicio.
        - r(Number)             : Tipo de interés libre de riesgo.
        - sigma (Number)        : riesgo o volatilidad del activo.
        - T (Number)            : Fecha de vencimiento.
        - d0 = 0 (Number)       : Valor de los dividendos
        - T0 = 0 (Number)       : Fecha inicial para el velor S0.
    Output:
        - P(Number)             : Valor de la Put.
    """
    from math import log,exp,sqrt
    from scipy.stats import norm

    d1 = (log(s0/K) + (r + (sigma**2)/(2)) * (T-T0)) / (sqrt(T-T0)*sigma)
    d2 = d1 - sigma*sqrt(T-T0)

    Nd1 = norm.cdf(-d1)
    Nd2 = norm.cdf(-d2)

    P = - s0*Nd1 + K*exp(-r*(T-T0))*Nd2 + d0
    return P

### Ambos valores para nuestro ejemplo.
Suponemos que tenemos una opción con precio inicial $s_0 = 74.625$, el precio del ejercicio $K =100$. La fecha de vencimiento es $1.6$ años, el tipo de interés libre de riesgo es $r=0.05$ anual y el riesgo $\sigma = 0.375$ anual.

In [12]:
s0 = 74.625
K = 100
T = 1.6
r = 0.05
sigma = 0.375

print( f"El valor de la Call: {valorCall(s0,K,r,sigma,T)}\nEl valor de la Put : {valorPut(s0,K,r,sigma,T)}")

El valor de la Call: 8.316364366583233
El valor de la Put : 26.002999005246814
